In [1]:
from abc import ABC

from gulpio2 import GulpDirectory
from pathlib import Path
from moviepy.editor import ImageSequenceClip, clips_array

from collections import defaultdict

import pickle
import pandas as pd
from pathlib import Path

import torch as t
import torch.nn as nn
import numpy as np
from omegaconf import OmegaConf
from typing import Any, Dict, List, Sequence, Union

from systems import EpicActionRecognitionSystem
from systems import EpicActionRecogintionDataModule

from utils.metrics import compute_metrics
from utils.actions import action_id_from_verb_noun
from scipy.special import softmax

from GPUtil import showUtilization as gpu_usage
from tqdm import tqdm

In [4]:
verb_noun_link = {}

rgb_train = GulpDirectory('../datasets/epic/gulp/rgb/p01_rgb_test/')


for i, c in enumerate(tqdm(rgb_train)):
    for _, meta in c:
        if meta['verb'] in verb_noun_link:
            verb_noun_link[meta['verb']].append(meta['noun'])
        else:
            verb_noun_link[meta['verb']] = [meta['noun']] 

56it [09:08,  9.80s/it]


In [10]:
xd = list(set(verb_noun_link['take']))

with open('../datasets/epic/labels/verb_noun.pkl', 'wb') as f:
    pickle.dump(verb_noun_link, f)

with open('../datasets/epic/labels/verb_noun_unique.pkl', 'wb') as f:
    pickle.dump({verb: list(set(verb_noun_link[verb])) for verb in verb_noun_link.keys()} , f)


In [17]:
with open('../datasets/epic/labels/verb_noun_unique.pkl', 'rb') as pkl:
    verb_noun_unique = pickle.load(pkl)
    

In [18]:
verb_noun_unique

{'open': ['oven',
  'bag:mozzarella',
  'bin:garbage',
  'bin:food',
  'cooker:rice',
  'kettle',
  'can',
  'container:tofu',
  'liquid:washing:up',
  'purse',
  'box:sausage',
  'bag',
  'rosemary',
  'bag:cereal',
  'freezer',
  'jar',
  'box:tomato',
  'box:cereal',
  'drawer',
  'box:egg',
  'milk:soy',
  'cooker:steam',
  'fridge',
  'oil:olive',
  'box:fruit',
  'box',
  'bin:recycling',
  'cupboard',
  'container',
  'rucksack',
  'box:mushroom',
  'lid',
  'top',
  'door',
  'bag:noodle',
  'oil',
  'olive',
  'bin',
  'rice',
  'milk:soya',
  'salt',
  'stock',
  'powder:curry',
  'bottle'],
 'turn-on': ['top', 'tap', 'oven', 'light', 'candle', 'kettle', 'hob'],
 'take': ['cloth:table',
  'basil',
  'end:aubergine',
  'bit:onion',
  'pan',
  'kettle',
  'can',
  'liquid:washing:up',
  'board:eating',
  'food',
  'board:cutting',
  'holder:pot',
  'bowl',
  'box:fruit',
  'oil:olive',
  'lettuce',
  'bag:bread',
  'lid',
  'yeast',
  'fork',
  'tray',
  'bin',
  'milk:soya',
 